In [7]:
#importing pandas for data
import pandas as pd
import math

#node class for tree
class Node:
  def __init__(self, left,middle, right,data,state):
    self.left= left
    self.middle = middle
    self.right = right
    self.data = data
    self.state = state


#this class creates a tree using the ID3 algorithm
class ID3:

  #constructor with variables training data, test data, sizes of each, and root of tree
  def __init__(self,dataset=None):
    split = round(958 * 1)
    self.trainingSize = split
    self.TestSize = 958 - split
    self.data = dataset[:split]
    self.dataTest = dataset[split:]
    #self.usedAttributes = {'topl':None,'topm':None,'topr':None,'midl':None,'midm':None,'midr':None,'botl':None,'botm':None,'botr':None} 
    #self.remainingAttributes =dataset
    #self.usedAttributes = []
    self.root = None
  
  #creates the tree using the ID3 algo 
  def CreateTree(self,usedAttributes, remainingAttributes):
    positive = negative = -1
    #checks if all attributes have been used
    if len(usedAttributes) > 0: 
      positive, negative = self.CountUsedAttributes(usedAttributes)
    #checks if there are only negative results
    if (positive == 0):
      return Node(None,None,None,"final",usedAttributes)
    #checks if there are only positive results
    if (negative == 0):
      return Node(None,None,None,"final",usedAttributes)
    #if there is only 1 remaining attribute, create node and split tree
    if (len(remainingAttributes) == 1):
      leftUsed = usedAttributes.copy()
      rightUsed = usedAttributes.copy()
      middleUsed =  usedAttributes.copy()
      leftUsed[remainingAttributes[0]] = "x"
      middleUsed[remainingAttributes[0]] = "b"
      rightUsed[remainingAttributes[0]] = "o"
      leftNode = Node(None,None,None,"final",leftUsed)
      midNode = Node(None,None,None,"final",middleUsed)
      rightNode = Node(None,None,None,"final",rightUsed)
      currentNode = Node(leftNode, midNode,rightNode, remainingAttributes[0],usedAttributes)
      print(remainingAttributes[0],usedAttributes)
      return currentNode;
    #otherwise calculates best infogain and then creates node
    else:
      infoGains = self.IG(usedAttributes.copy(), remainingAttributes.copy())
      currentNode = Node(None, None, None, infoGains,usedAttributes.copy())
      #assigns root
      if (len(remainingAttributes) == 9):
        self.root = currentNode

      #preparations to create next nodes
      remainingAttributes.remove(infoGains)
      leftUsed = usedAttributes.copy()
      rightUsed = usedAttributes.copy()
      middleUsed =  usedAttributes.copy()
      leftUsed[infoGains] = "x"
      middleUsed[infoGains] = "b"
      rightUsed[infoGains] = "o"

      #Printing to help show progress of tree
      if (len(remainingAttributes) == 8):
        print("Beginning tree creation")
      #recursively creats left side of tree
      currentNode.left = self.CreateTree(leftUsed.copy(), remainingAttributes.copy())

      #Printing to help show progress of tree
      if (len(remainingAttributes) == 8):
        print("33% way through")

      #recursively creats middle side of tree
      currentNode.middle = self.CreateTree(middleUsed.copy(),remainingAttributes.copy())

      #Printing to help show progress of tree
      if (len(remainingAttributes) == 8):
        print("66% way through")

      #recursively creats right side of tree
      currentNode.right = self.CreateTree(rightUsed.copy(), remainingAttributes.copy())

      #returns node
      return currentNode;
  #this itterates through the data to count number of matching data points to the current state of the tree
  def CountUsedAttributes(self,UsedAttributes):
    skip = True
    for attr in UsedAttributes:
      if UsedAttributes[attr] != None:
        skip = False
    if skip:
      return 1,1;
    negative = positive =0
    for x in self.data.iterrows():
      matching = True 
      for attr in UsedAttributes:
        if UsedAttributes[attr] != x[1][attr] and UsedAttributes[attr] != None:
          matching = False
          break;
      if matching:
          if "positive" == x[1]["result"]:
            positive += 1
          else:
            negative += 1
    return positive, negative;
  
  #this calculates infogain of each potential remainingAttribute and then will return the one with the best infogain
  def IG(self, usedAttributes,remainingAttributes):
    highestInfo = 1000
    bestChild = ""
    for attr in remainingAttributes:
      if usedAttributes != None:
        leftUsed = usedAttributes.copy()
        rightUsed = usedAttributes.copy()
        middleUsed =  usedAttributes.copy()
        leftUsed[attr] = "x"
        middleUsed[attr] = "b"
        rightUsed[attr] = "o"
        leftPos, leftNeg = self.CountUsedAttributes(leftUsed)
        midPos, midNeg = self.CountUsedAttributes(middleUsed)
        rightPos, rightNeg = self.CountUsedAttributes(rightUsed)
        leftEnt = self.Ent(leftPos,leftNeg)
        midEnt = self.Ent(midPos,midNeg)
        rightEnt = self.Ent(rightPos,rightNeg)
        total = leftPos + leftNeg + midPos + midNeg + rightPos + rightNeg
        leftTotal = leftPos + leftNeg
        midTotal = midPos + midNeg
        rightTotal = rightPos + rightNeg
        totalEnt = (leftTotal/total)*leftEnt + (midTotal/total)*midEnt + (rightTotal/total)*rightEnt
        if (totalEnt < highestInfo):
          highestInfo = totalEnt
          bestChild = attr
    return bestChild

  #entropy calculator
  def Ent(self, pos,neg):
    if (pos > 0 and neg > 0):
      return -1*(pos/(pos+neg))*math.log2((pos/(pos+neg))) - (neg/(pos+neg))*math.log2((neg/(pos+neg)))
    else:
      return 0

  #used to check if tree is correctly created
  def results(self):
    print("Most important square: ", self.root.data)
    print("Database size: 958, Test Data size:",self.TestSize,", Training data size: ",self.trainingSize)
    print("Prediction Accuracy on entire dataset: ", self.Accuracy(self.data)/958 * 100)    

  #used to calculate accuracy of tree
  def Accuracy(self, data):
    root = self.root
    matching = 0
    for x in data.iterrows():
      currentNode = root
      while currentNode != None:
        if currentNode.data == "final":
          positive, negative = self.CountUsedAttributes(currentNode.state)
          if positive == 0:
            if x[1]["result"] == "negative":
              matching += 1
              break;
          if negative == 0:
            if x[1]["result"] == "positive":
              matching += 1
              break;
          break;
        else:
          if x[1][currentNode.data] == "x":
            currentNode = currentNode.left
          elif x[1][currentNode.data] == "o":
            currentNode = currentNode.right
          elif x[1][currentNode.data] == "b":
            currentNode = currentNode.middle    
    return matching

  #decides next move of the bot to play tictactoe
  def NextMove(self,root,currentState,moveCount,data):
      moves = 0
      currentNode = root
      while moves != moveCount:
        if currentNode.data == "final":
          currentState, finished = self.WinningMove(currentState,data)
          return currentState, finished
        if currentState[currentNode.data] == "x":
          moves +=1
          currentNode = currentNode.left
        elif currentState[currentNode.data] == "o":
          moves +=1
          currentNode = currentNode.right
        elif currentState[currentNode.data] == "b":
          currentNode = currentNode.middle
      currentState[currentNode.data] = "x"
      return currentState, True

  #decides winning move because bot is unable to decide using ID3 algorithm
  def WinningMove(self,currentState,data):
    bMax = 0
    bestX = ""
    for x in data.iterrows():
      bCount = 0
      matching = True
      for attr in currentState:
        if currentState[attr] == "x" and "x" != x[1][attr]:
          matching = False
          break
        if currentState[attr] == "o" and "o" != x[1][attr]:
          matching = False
          break
        if currentState[attr] == "b" and "x" == x[1][attr]:
          tempState = attr
        if x[1][attr] == "b":
          bCount += 1
      if matching and bCount > bMax:
        bMax = bCount
        bestX = tempState
    currentState[tempState] = "x"

    for x in data.iterrows():
      completeMatching = True
      for attr in currentState:
        if currentState[attr] != x[1][attr]:
            completeMatching = False
      if completeMatching:
          break
    return currentState, not completeMatching

#gets data using pandas, then creates the tree, then plays tic tac toe with user 
def main():
  header=['topl','topm','topr','midl','midm','midr','botl','botm','botr',"result"]
  df = pd.read_csv("https://personal.utdallas.edu/~btr180000/ticktacktoe.data",header=None,names=header)
  initAlgo = ID3(df)
  remain = ['topl','topm','topr','midl','midm','midr','botl','botm','botr']
  empty = {}
  state = {'topl':None,'topm':None,'topr':None,'midl':None,'midm':None,'midr':None,'botl':None,'botm':None,'botr':None}
  root = initAlgo.CreateTree(state,remain)

  state = {'topl':"b",'topm':"b",'topr':"b",'midl':"b",'midm':"b",'midr':"b",'botl':"b",'botm':"b",'botr':"b"}
  squares = ["1","2","3","4","5","6","7","8","9"]
  playing = True
  print("__1__|__2__|__3__")
  print("__4__|__5__|__6__")
  print("  7  |  8  |  9  \n")
  moveCount = 0
  print("Enter a number corresponding to the square on the first board.You are playing as o ")

  while (playing):
    state,playing = initAlgo.NextMove(root,state,moveCount,df)
    moveCount += 1
    print("The bot has made a move!")
    print("__"+state["topl"]+"__|__"+state["topm"]+"__|__"+state["topr"]+"__")
    print("__"+state["midl"]+"__|__"+state["midm"]+"__|__"+state["midr"]+"__")
    print("  "+state["botl"]+"  |  "+state["botm"]+"  |  "+state["botr"]+"  \n")
    if not playing and moveCount != 9:
      print("The bot has won!")
      break;
    if moveCount == 9:
      print("Game has tied!")
      break;
    inp = input("Input: ")
    if (state[list(state)[int(inp)-1]] != "b"):
      print("invalid input :( game is ending")
    squares[int(inp) - 1] = "o"
    state[list(state)[int(inp)-1]] = "o"
    moveCount += 1
    print("__"+state["topl"]+"__|__"+state["topm"]+"__|__"+state["topr"]+"__")
    print("__"+state["midl"]+"__|__"+state["midm"]+"__|__"+state["midr"]+"__")
    print("  "+state["botl"]+"  |  "+state["botm"]+"  |  "+state["botr"]+"  \n")

    pos,neg = initAlgo.CountUsedAttributes(state)
    if (pos == 0 and neg != 0):
      print("User has won!")
      break;
      
if __name__ == "__main__":
  main()

Beginning tree creation
33% way through
66% way through
__1__|__2__|__3__
__4__|__5__|__6__
  7  |  8  |  9  

Enter a number corresponding to the square on the first board.You are playing as o 
The bot has made a move!
__b__|__b__|__b__
__b__|__x__|__b__
  b  |  b  |  b  

__b__|__b__|__b__
__b__|__x__|__b__
  o  |  b  |  b  

The bot has made a move!
__b__|__b__|__b__
__b__|__x__|__b__
  o  |  b  |  x  

__o__|__b__|__b__
__b__|__x__|__b__
  o  |  b  |  x  

The bot has made a move!
__o__|__b__|__b__
__b__|__x__|__b__
  o  |  x  |  x  

__o__|__b__|__b__
__o__|__x__|__b__
  o  |  x  |  x  

User has won!
